In [1]:
import fsspec
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

# return all the COGS for a given year
def get_tifs(last_prefix):
    files = fs.ls(f'eco-w1/in0/{last_prefix}')
    return [f for f in files if f.endswith('tif')]


tif_list = get_tifs('rapv2')

tif_list[0:4]



['eco-w1/in0/rapv2/vegetation-cover-v2-1984.tif',
 'eco-w1/in0/rapv2/vegetation-cover-v2-1985.tif',
 'eco-w1/in0/rapv2/vegetation-cover-v2-1986.tif',
 'eco-w1/in0/rapv2/vegetation-cover-v2-1987.tif']

In [2]:
import rasterio
import xarray as xr

#returns an xarray DA
def overview_ugly_xr_read(s3_tif_object):
    da = xr.open_rasterio(f's3://{s3_tif_object}')
    src = rasterio.open(f's3://{s3_tif_object}', 'r')
    #nsub = 4  # original scale factor
    #nsub = 64
    nsub=128
    nsub_half = int(nsub/2)
    data = src.read(out_shape=(1, int(src.height / nsub), int(src.width / nsub)))
    DA = xr.DataArray(data, dims=da.dims, coords=[da.band, da.y[nsub_half:-nsub_half:nsub], da.x[nsub_half:-nsub_half:nsub]])
    DA.attrs = da.attrs
    return(DA)

In [3]:
years = range(1984,2019+1)
years

range(1984, 2020)

In [4]:
import pickle

In [ ]:
%%time

howmany = len(years)
dalist =[overview_ugly_xr_read(tif) for tif in tif_list[:howmany]]

da = xr.concat(dalist, dim='band')
da = da.rename({'band':'year'})
da = da.assign_coords(year=years)

with open('rap_lzw_1984_2019.pickle', 'wb') as handle:
    pickle.dump(da, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
da = da.assign_coords(year=years)

In [ ]:
#da.dims
da.coords

In [ ]:
import hvplot.xarray

img_figure = da.hvplot.image(dynamic=False,rasterize=True,grid=True, clim=(0,50),
                             invert=False, width=650, height=500, cmap='OrRd')

In [ ]:
img_figure.hist()

In [ ]:
da.dims